In [ ]:
# Code source: Sebastian Curi and Andreas Krause, based on Jaques Grobler (sklearn demos).
# License: BSD 3 clause

# We start importing some modules and running some magic commands
%matplotlib inline
%reload_ext autoreload
%load_ext autoreload
%autoreload 2

# General math and plotting modules.
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt

# Project files.
from utilities.util import gradient_descent
from utilities.load_data import polynomial_data
from utilities import plot_helpers
from utilities.regressors import LinearRegressor
from utilities.regularizers import Regularizer, L2Regularizer

# Widget and formatting modules
import ipywidgets
from ipywidgets import interact, interactive, interact_manual, fixed
import pylab
# If in your browser the figures are not nicely vizualized, change the following line. 
pylab.rcParams['figure.figsize'] = (10, 5)

# Machine Learning library. 
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import cross_val_score


In [ ]:
def true_fun(X):
    return np.cos(1.5 * np.pi * X)

def bias_variance_to(n_samples, degree, noise):
    np.random.seed(0)

    X = np.sort(np.random.rand(n_samples))
    y = true_fun(X) + np.random.randn(n_samples) * noise

    polynomial_features = PolynomialFeatures(degree=degree,
                                             include_bias=True)
    linear_regression = LinearRegression()
    pipeline = Pipeline([("polynomial_features", polynomial_features),
                         ("linear_regression", linear_regression)])
    pipeline.fit(X[:, np.newaxis], y)

    # Evaluate the models using crossvalidation
    scores = cross_val_score(pipeline, X[:, np.newaxis], y,
                             scoring="neg_mean_squared_error", cv=10)

    X_test = np.linspace(-1, 2, 100)
    plt.plot(X_test, pipeline.predict(X_test[:, np.newaxis]), label="Model")
    plt.plot(X_test, true_fun(X_test), label="True function")
    plt.scatter(X, y, edgecolor='b', s=20, label="Samples")
    plt.xlabel("x")
    plt.ylabel("y")
    plt.xlim((-1, 2))
    plt.ylim((-2, 2))
    plt.legend(loc="best")
    plt.title("Degree {}\nMSE = {:.2e}(+/- {:.2e})".format(
        degree, -scores.mean(), scores.std()))
    plt.show()
    
interact(bias_variance_to, 
        n_samples=ipywidgets.IntSlider(value=30,
                                         min=30,
                                         max=300,
                                         step=1,
                                         description='Number of samples:',
                                         style={'description_width': 'initial'},
                                         continuous_update=False),
        degree=ipywidgets.IntSlider(value=1,
                                         min=1,
                                         max=15,
                                         step=1,
                                         description='Polynomial Degree:',
                                         style={'description_width': 'initial'},
                                         continuous_update=False),
         noise=ipywidgets.FloatSlider(value=0.1,
                                      min=0,
                                      max=1,
                                      step=0.1,
                                      readout_format='.1f',
                                      description='Noise level:',
                                      style={'description_width': 'initial'},
                                      continuous_update=False),);

In [ ]:
degrees = np.arange(1, 10, 1)
def bias_variance_to(n_samples, noise):
    n_samples = int(n_samples)
    np.random.seed(0)
    score = []
    bias = []
    variance = []

    for degree in degrees:
        X = np.sort(np.random.rand(n_samples))
        y = true_fun(X) + np.random.randn(n_samples) * noise

        polynomial_features = PolynomialFeatures(degree=degree,
                                                 include_bias=True)
        linear_regression = LinearRegression()
        pipeline = Pipeline([("polynomial_features", polynomial_features),
                             ("linear_regression", linear_regression)])
        pipeline.fit(X[:, np.newaxis], y)

        # Evaluate the models using crossvalidation
        scores = cross_val_score(pipeline, X[:, np.newaxis], y,
                                 scoring="neg_mean_squared_error", cv=5)
        
        # Estimate bias
        h_star = true_fun(X) 
        pipeline.fit(X[:, np.newaxis], h_star)
        bias_scores = cross_val_score(pipeline, X[:, np.newaxis], h_star, scoring="neg_mean_squared_error", cv=5)
            
        score.append(-scores.mean())
        bias.append(-bias_scores.mean())
        
        # Estimate Variance
        variance.append(score[-1] - bias[-1] - noise ** 2)
        

    plt.plot(degrees, score, label='Total')
    plt.plot(degrees, bias, label='Bias')
    plt.plot(degrees, variance, label='Variance')

    plt.plot(degrees, noise ** 2 * np.ones_like(degrees), '--', label='Noise')
    plt.legend(loc='upper left')
    plt.ylabel('MSE')
    plt.xlabel('Polynomial degree (Model Complexity)')
    plt.ylim([0, 4])
interact(bias_variance_to,
#          n_samples=[50, 500, 1000, 10000],
         n_samples=ipywidgets.IntText(value=500, step=300, description='Samples:', continuous_update=False),
         noise=ipywidgets.FloatSlider(value=0.1, min=0, max=1, step=0.1,
                                      readout_format='.1f',
                                      description='Noise:',
#                                       style={'description_width': 'initial'},
                                      continuous_update=False),);